# Text Sentiment Analysis of IMDB Movie reviews using NLP (Word2Vec and RNN) for MYM Intern Assesment

## Assesment Objectives: 
### - Preprocessing the data
### - Converting Text(words) to Vectors using word2vec 
### - Using the word representations given by word2vec to feed a RNN and training the model
### - Testing the model


In [24]:
!pip freeze | grep gensim ##Checking the version of Gensim - Word2Vec

gensim==4.2.0


## The Data

### Starting with 30% of the sentences from TensorFlow Datasets of IMDB reviews to check the RAM compatibility of the PC to train the model faster by splitting the datasets as X_train, y_train, X_test and y_test.
### Then preprocessing the textual data to create input features for a natural language processing (NLP) model.



In [49]:
import numpy as np
import pandas as pd
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
      assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
      len_train = int(percentage_of_sentences/100*len(train_sentences))
      train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
  
      len_test = int(percentage_of_sentences/100*len(test_sentences))
      test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]
    
    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=30)

In [50]:
y_test

array([1, 1, 0, ..., 1, 1, 0])

## First, training a word2vec model (with the arguments that we want) on your training sentence. Store it into the `word2vec` variable.

In [26]:
from gensim.models import Word2Vec

word2vec = Word2Vec(sentences=X_train, vector_size=60, min_count=10, window=10)

In [55]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

# Embed the training and test sentences
X_train_embed = embedding(word2vec, X_train)
X_test_embed = embedding(word2vec, X_test)


# Pad the training and test embedded sentences
X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=200)
X_test_pad = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=200)

### It's a good practice to check check the following for `X_train_pad` and `X_test_pad`:
#### - they are numpy arrays
#### - they are 3-dimensional
#### - the last dimension is of the size of your word2vec embedding space (you can get it with `word2vec.wv.vector_size`\\
#### - the first dimension is of the size of your `X_train` and `X_test`

In [56]:
for X in [X_train_pad, X_test_pad]:
    assert type(X) == np.ndarray
    assert X.shape[-1] == word2vec.wv.vector_size


assert X_train_pad.shape[0] == len(X_train)
assert X_test_pad.shape[0] == len(X_test)

## Baseline Model

In [57]:
# It is always good to have a very simple model to test your own model against
# Baseline accuracy can be to predict the label that is the most present in `y_train`.
from sklearn.metrics import accuracy_score

unique, counts = np.unique(y_train, return_counts=True)
counts = dict(zip(unique, counts))
print('Number of labels in train set', counts)

y_pred = 0 if counts[0] > counts[1] else 1

print('Baseline accuracy: ', accuracy_score(y_test, [y_pred]*len(y_test)))

Number of labels in train set {0: 3739, 1: 3761}
Baseline accuracy:  0.49173333333333336


## The Model

In [58]:
from tensorflow.keras import Sequential
from tensorflow.keras import layers

# writing a RNN model with Masking, LSTM and Dense layers.

def init_model():
    model = Sequential()
    model.add(layers.Masking())
    model.add(layers.LSTM(20, activation='tanh'))
    model.add(layers.Dense(15, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', #compiling the model with rmsprop optimizer
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    
    return model

model = init_model()

In [59]:
#  Fiting the model on embedded and padded data with the early stopping criterion.

from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=5, restore_best_weights=True)

model.fit(X_train_pad, y_train, 
          batch_size = 32,
          epochs=100,
          validation_split=0.3,
          callbacks=[es]
         )

Epoch 1/100
165/165 [==============================] - 10s 45ms/step - loss: 0.6769 - accuracy: 0.5598 - val_loss: 0.6495 - val_accuracy: 0.6378
Epoch 2/100
165/165 [==============================] - 5s 32ms/step - loss: 0.6083 - accuracy: 0.6653 - val_loss: 0.5855 - val_accuracy: 0.6916
Epoch 3/100
165/165 [==============================] - 5s 32ms/step - loss: 0.5348 - accuracy: 0.7326 - val_loss: 0.5118 - val_accuracy: 0.7498
Epoch 4/100
165/165 [==============================] - 5s 32ms/step - loss: 0.4917 - accuracy: 0.7684 - val_loss: 0.5167 - val_accuracy: 0.7542
Epoch 5/100
165/165 [==============================] - 5s 32ms/step - loss: 0.4614 - accuracy: 0.7810 - val_loss: 0.4722 - val_accuracy: 0.7956
Epoch 6/100
165/165 [==============================] - 5s 32ms/step - loss: 0.4503 - accuracy: 0.7870 - val_loss: 0.4606 - val_accuracy: 0.7867
Epoch 7/100
165/165 [==============================] - 5s 33ms/step - loss: 0.4242 - accuracy: 0.8070 - val_loss: 0.5197 - val_accuracy

In [60]:
# Evaluating the model on the test set.

result = model.evaluate(X_test_pad, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {result[1]*100:.3f}%')

The accuracy evaluated on the test set is of 80.427%


## Trained Word2Vec - Transfer Learning


### The accuracy of the above the baseline model, might be quite low. By improving the quality of the embedding we can Improve accuracy of the model.

### Let's improve the quality of our embedding, instead of just loading a larger corpus, let's benefit from the embedding that others have learned. Because, the quality of an embedding, i.e. the proximity of the words, can be derived from different tasks. This is exactly what transfer learning is.

### Listing all the different models available in the word2vec using gensim api.

In [61]:
import gensim.downloader as api
print(list(api.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [62]:
word2vec_transfer = api.load("glove-wiki-gigaword-100")

[==================================================] 100.0% 128.1/128.1MB downloaded


In [66]:
print(len(word2vec_transfer.key_to_index))
print(len(word2vec_transfer['dog']))

400000
100


In [67]:
# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

# Embed the training and test sentences
X_train_embed_2 = embedding(word2vec_transfer, X_train)
X_test_embed_2 = embedding(word2vec_transfer, X_test)

In [68]:
# Pad the training and test embedded sentences
X_train_pad_2 = pad_sequences(X_train_embed_2, dtype='float32', padding='post', maxlen=200)
X_test_pad_2 = pad_sequences(X_test_embed_2, dtype='float32', padding='post', maxlen=200)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

es = EarlyStopping(patience=5, restore_best_weights=True)

model = init_model()

model.fit(X_train_pad_2, y_train, 
          batch_size = 32,
          epochs=10,
          validation_split=0.3,
          callbacks=[es]
         )
model.save('my_model')

Epoch 1/10
165/165 [==============================] - 9s 39ms/step - loss: 0.6701 - accuracy: 0.5792 - val_loss: 0.6811 - val_accuracy: 0.5671
Epoch 2/10
165/165 [==============================] - 6s 34ms/step - loss: 0.6194 - accuracy: 0.6568 - val_loss: 0.7026 - val_accuracy: 0.5778
Epoch 3/10
165/165 [==============================] - 6s 34ms/step - loss: 0.5673 - accuracy: 0.7110 - val_loss: 0.5895 - val_accuracy: 0.6907
Epoch 4/10
165/165 [==============================] - 6s 34ms/step - loss: 0.5250 - accuracy: 0.7505 - val_loss: 0.6206 - val_accuracy: 0.6889
Epoch 5/10
165/165 [==============================] - 6s 34ms/step - loss: 0.5015 - accuracy: 0.7653 - val_loss: 0.4784 - val_accuracy: 0.7844
Epoch 6/10
165/165 [==============================] - 6s 35ms/step - loss: 0.4792 - accuracy: 0.7766 - val_loss: 0.4622 - val_accuracy: 0.7902
Epoch 7/10
165/165 [==============================] - 6s 34ms/step - loss: 0.4462 - accuracy: 0.8011 - val_loss: 0.5971 - val_accuracy: 0.7102

In [43]:
res = model.evaluate(X_test_pad_2, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

The accuracy evaluated on the test set is of 78.213%
